In [1]:
import numpy as np
from copy import copy

In [6]:
def simulate_episode(init_prob_matrix, n_steps_max):
  prob_matrix = init_prob_matrix.copy()
  n_nodes = prob_matrix.shape[0]
  # the probality that a node is active at the beginning is given by a
  # binomial distribution with p = 0.1
  initial_active_nodes = np.random.binomial(1, 0.1, size=(n_nodes))
  # the matrix that holds the active nodes for each episode
  history = np.array([initial_active_nodes])
  # active_nodes keeps track of the active nodes at the current episode,
  # it's initialized with the initial active nodes
  active_nodes = initial_active_nodes
  newly_active_nodes = active_nodes
  t = 0
  while(t<n_steps_max and np.sum(newly_active_nodes)>0):
    # p is a n_nodes x n_nodes probability matrix where only the
    # columns corresponding to the active nodes are not zero
    p = (prob_matrix.T * active_nodes).T
    activated_edges = p > np.random.rand(p.shape[0], p.shape[1])
    # we remove from the values of the probability matrix all the
    # values of previously activated nodes   
    prob_matrix = prob_matrix * ((p!=0)==activated_edges)
    # the new activated nodes are the ones in which at least on
    # of its edges has been activated in the previous step     
    newly_active_nodes = (np.sum(activated_edges, axis=0) > 0) * (1 - active_nodes)
    active_nodes = np.array(active_nodes + newly_active_nodes)
    history = np.concatenate((history, [newly_active_nodes]), axis=0)
    t+=1
  return history

In [3]:
def estimate_probabilities(dataset, node_index, n_nodes):
  # we start by assigning equal probability to each edge 
  estimated_prob = np.ones(n_nodes)*1.0/(n_nodes - 1)
  credits = np.zeros(n_nodes)
  # stores the occurences of each node in each episode
  occur_v_active = np.zeros(n_nodes)
  n_episodes = len(dataset)

  for episode in dataset:
    # store in which timestamp the w node has been active
    idx_w_active = np.argwhere(episode[:, node_index].reshape(-1))
    # we check which where the nodes active in the previous step
    if len(idx_w_active) > 0 and idx_w_active > 0:
      active_nodes_in_prev_step = episode[idx_w_active - 1, :].reshape(-1)
      credits += active_nodes_in_prev_step/np.sum(active_nodes_in_prev_step)
    # we check the occurences of every node
    for v in range(0, n_nodes):
      if(v!=node_index):
        idx_v_active = np.argwhere(episode[:, v] == 1).reshape(-1)
        if len(idx_v_active)>0 and (idx_v_active<idx_w_active or len(idx_w_active)==0):
          occur_v_active[v]+=1

  estimated_prob = credits/occur_v_active
  estimated_prob = np.nan_to_num(estimated_prob)
  return estimated_prob

In [7]:
n_nodes = 5
n_episodes = 1000
prob_matrix = np.random.uniform(0.0, 0.1, (n_nodes, n_nodes))
node_index = 4
dataset = []

for e in range(0, n_episodes):
  dataset.append(simulate_episode(init_prob_matrix=prob_matrix, n_steps_max=10))

estimated_prob = estimate_probabilities(dataset=dataset, node_index=node_index, n_nodes=n_nodes)

print("True P Matrix", prob_matrix[:,4])
print("Estimated P Matrix", estimated_prob)

True P Matrix [0.0277947  0.01018559 0.00809074 0.06230941 0.08101648]
Estimated P Matrix [0.02951389 0.00609756 0.01257862 0.08250825 0.        ]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
